______________________________________
# <center>**Trabajo Practico Nº1 para la Materia *Organización de Datos***</center>

*Integrantes*:
- 103963	Carolina Di Matteo	cdimatteo@fi.uba.ar
- 101231	Pablo Salvador Dimartino	pdimartino@fi.uba.ar
- 100113	Juan Sebastian Burgos	jsburgos@fi.uba.ar
- 104415	Valentina Laura Correa	vcorrea@fi.uba.ar

*Grupo*: 14

*Repositorio*: [github](https://github.com/valencorrea/7506R-2C2022-GRUPO14)

*Curso*: Rodriguez

*Cuatrimestre*: 2c2022

Datos provistos por [properati](https://www.properati.com.ar).
______________________________________


In [77]:
#Importación de librerías

import numpy as np
import cmath as math
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

#Configuración de Warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

#Ejecución con Drive
if 'google.colab' in str(get_ipython()):
    IN_COLAB = True
else:
    IN_COLAB = False

if IN_COLAB :
    from google.colab import drive
    drive.mount('/content/drive')
    properati = pd.read_csv('/content/drive/MyDrive/📔 Organización de Datos (75.06)/TPS/TP1/properati_argentina_2021.csv')
    properati_decrip = pd.read_csv('/content/drive/MyDrive/📔 Organización de Datos (75.06)/TPS/TP1/properati_argentina_2021_decrip.csv')
else:
    properati=pd.read_csv('properati_argentina_2021.csv')
    properati_decrip = pd.read_csv('properati_argentina_2021_decrip.csv')

In [78]:
df_decrip = properati_decrip.head(10000).copy()
df_decrip['property_description'].isna().sum()
properati_decrip

,id,property_description
0,BZCeiGkJr1WBUi6pKJQwJQ==,Corredor Responsable: MARIA ALEJANDRA GENOVEVA...
1,zuHOGgUE0UB71W7m/YCLoA==,¡HERMOSO PH TIPO CASA DE 2 AMBIENTES!<br>APTO ...
2,TUL813tXchVBlHUsfaA6DA==,Excelente PH en Núñez 4 amb!!!! Oportunidad !!...
3,gn55JPmcjftBV/at1a7fPg==,"UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO,..."
4,qg4m3QHcuo1+wMmyp7kx3Q==,Ph 2 Amb. Villa Crespo exclusivo uso profesion...
...,...,...
460149,MbQ4rqEzjp9Jfc3Sywb/YQ==,Corredor Responsable: Hector Garaygorta / Cris...
460150,fbuG1ohYC623nxLYPQPBvw==,Corredor Responsable: MARIA DEL ROSARIO NEME -...
460151,Ry5hMwctFXD7rLoUpzduxQ==,En cumplimiento de las leyes provinciales vige...
460152,xH37f+l+VOP8Vj0T3g+r6g==,En cumplimiento de las leyes provinciales vige...


In [79]:
df_decrip.dropna(inplace=True)

In [80]:
from collections import Counter
Counter(" ".join(df_decrip["property_description"]).split()).most_common(100)


[('de', 87457),
 ('y', 51602),
 ('con', 44219),
 ('en', 29525),
 ('la', 27091),
 ('a', 23099),
 ('el', 15988),
 ('del', 13754),
 ('por', 12384),
 ('al', 11652),
 ('-', 11217),
 ('que', 10840),
 ('un', 10345),
 ('para', 9347),
 ('los', 9059),
 ('las', 8638),
 ('se', 7161),
 ('una', 6307),
 ('DE', 6104),
 ('2', 6068),
 ('son', 5305),
 ('baño', 4857),
 ('cocina', 4564),
 ('3', 4201),
 ('comedor', 4077),
 ('es', 4042),
 ('dos', 3771),
 ('<br>', 3662),
 ('Y', 3337),
 ('cuenta', 3313),
 ('no', 3290),
 ('CON', 3247),
 ('o', 3218),
 (',', 3209),
 ('x', 3166),
 ('muy', 3136),
 ('EN', 3012),
 ('1', 2974),
 ('living', 2958),
 ('propiedad', 2934),
 ('casa', 2798),
 ('A', 2786),
 ('dormitorios', 2765),
 ('ambientes', 2686),
 ('piso', 2675),
 ('sobre', 2659),
 ('esta', 2613),
 ('encuentra', 2520),
 ('La', 2437),
 ('4', 2398),
 ('medidas', 2372),
 ('El', 2343),
 ('completo', 2321),
 ('dormitorio', 2296),
 ('/', 2294),
 ('planta', 2267),
 ('valor', 2247),
 ('amplio', 2197),
 ('departamento', 2185),
 (

In [81]:
df_decrip["property_description"]

0       Corredor Responsable: MARIA ALEJANDRA GENOVEVA...
1       ¡HERMOSO PH TIPO CASA DE 2 AMBIENTES!<br>APTO ...
2       Excelente PH en Núñez 4 amb!!!! Oportunidad !!...
3       UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO,...
4       Ph 2 Amb. Villa Crespo exclusivo uso profesion...
                              ...                        
9995    EN VENTA:<br><br>Departamento de 1 Ambiente en...
9996    Impecable departamento de 3 ambientes, con dos...
9997    Venta  Departamento 2 ambientes ,Muy luminoso....
9998    Departamento tipo ph 2 ambientes  A ESTRENAR e...
9999    Departamento 3 ambientes planta baja por pasil...
Name: property_description, Length: 9994, dtype: object

In [82]:
df_decrip["property_description"] = df_decrip["property_description"].apply(lambda line: line.replace("<br>", " ")) # remover etiqueta de html
df_decrip["property_description"] = df_decrip["property_description"].apply(lambda line: line.lower()) # Todo a minúsculas
import unidecode
df_decrip["property_description"] = df_decrip["property_description"].apply(lambda line: unidecode.unidecode(line)) # Sin tildes por posibles errores de gramática

In [83]:
import re
df_decrip["property_description"] = df_decrip["property_description"].apply(lambda line: re.sub("!+" , ". ", line)) #Remueve signos de exclamación
df_decrip["property_description"] = df_decrip["property_description"].apply(lambda line: re.sub(r'[^\w]', ' ', line)) # remover símbolos

In [84]:
df_decrip["property_description"] = df_decrip["property_description"].apply(lambda line: re.sub("\s\s+" , " ", line)) # remover multiples espacios para facilitar lectura

In [85]:
# export = df_decrip["property_description"].head()
# np.savetxt('export.txt', export.values, fmt='%s')

In [86]:
df_nostopwords = df_decrip.copy()

with open('stopwords') as f:
    lines = f.read().splitlines()

f = lambda x: ' '.join([item for item in x.split() if item not in lines])
df_nostopwords["property_description"] = df_nostopwords["property_description"].apply(f)

df_nostopwords["property_description"].head(10)


0    corredor responsable maria alejandra genoveva ...
1    hermoso ph tipo casa 2 ambientes apto credito ...
2    excelente ph nunez 4 amb oportunidad 154 m2 to...
3    unico ph 4 dormitorios impecable hermoso tranq...
4    ph 2 amb villa crespo exclusivo profesional ex...
5    vende ph interno ubicado excelente barrio vill...
6    propiedad ubicada gobernador alcorta cuadra co...
7    casa venta calle alejandro magno hermosa vista...
8    hermosa casa ph ubicado 10 cuadras centro vill...
9    hermosa casa ph ubicada barrio santa rita barr...
Name: property_description, dtype: object

In [87]:
from collections import Counter
Counter(" ".join(df_nostopwords["property_description"]).split()).most_common(100)

[('cocina', 9743),
 ('bano', 9211),
 ('2', 9101),
 ('comedor', 8234),
 ('living', 6949),
 ('3', 6934),
 ('1', 5772),
 ('dormitorios', 5298),
 ('propiedad', 5293),
 ('piso', 5172),
 ('completo', 5116),
 ('dormitorio', 5044),
 ('ambientes', 5007),
 ('casa', 4928),
 ('balcon', 4757),
 ('departamento', 4652),
 ('planta', 4424),
 ('m2', 4372),
 ('frente', 4365),
 ('4', 4206),
 ('excelente', 4043),
 ('amplio', 3802),
 ('pisos', 3620),
 ('edificio', 3600),
 ('x', 3568),
 ('expensas', 3475),
 ('lavadero', 3449),
 ('medidas', 3446),
 ('venta', 3194),
 ('cochera', 3101),
 ('av', 3003),
 ('ley', 2935),
 ('corredor', 2925),
 ('barrio', 2904),
 ('suite', 2859),
 ('vista', 2816),
 ('placard', 2804),
 ('c', 2770),
 ('mas', 2760),
 ('parrilla', 2747),
 ('inmueble', 2660),
 ('zona', 2615),
 ('alquiler', 2535),
 ('cuadras', 2473),
 ('servicios', 2451),
 ('5', 2434),
 ('metros', 2417),
 ('responsable', 2404),
 ('aire', 2393),
 ('principal', 2367),
 ('ubicacion', 2363),
 ('patio', 2342),
 ('salida', 2296)

Seleccionamos los aspectos que nos parecen relevantes, para luego buscar sus posibles valores

cocina, piso, balcon, frente, expensas, lavadero, cochera, placard, parrilla, aire, terraza, calefaccion

In [115]:
from numpy import NaN
from types import NoneType
aspectos = ['cocina', 'piso', 'calefaccion']

def f(line, word):
    format = r"\W*([\w]+)"
    n = 2
    x = re.search(r'{}\W*{}{}'.format(format*n, word, format*n), line)
    if not isinstance(x, NoneType):
        return x.group()
    else:
        return ""

for word in aspectos:
    df_nostopwords[word] = df_nostopwords["property_description"].apply(lambda line: f(line, word))

df_nostopwords.head(20)

,id,property_description,cocina,piso,calefaccion
0,BZCeiGkJr1WBUi6pKJQwJQ==,corredor responsable maria alejandra genoveva ...,completo hidromasaje cocina muebles mesada,aberturas cedro pisos pinotea,
1,zuHOGgUE0UB71W7m/YCLoA==,hermoso ph tipo casa 2 ambientes apto credito ...,pisos ceramicos cocina semi integrada,patio privado pisos ceramicos,
2,TUL813tXchVBlHUsfaA6DA==,excelente ph nunez 4 amb oportunidad 154 m2 to...,2 habitaciones cocina bano completo,zona consta piso 2 habitaciones,
3,gn55JPmcjftBV/at1a7fPg==,unico ph 4 dormitorios impecable hermoso tranq...,c extractor cocina comedor diario,desayunadora material piso dormitorio principal,
4,qg4m3QHcuo1+wMmyp7kx3Q==,ph 2 amb villa crespo exclusivo profesional ex...,pisos parquet cocina bacha acero,ambientes frente pisos parquet,
5,XztoBkv1vE+dGuId8GiPyA==,vende ph interno ubicado excelente barrio vill...,deposito banos cocina living comedor,,
6,JEhUB1WqHYnvkmuPWlDgmg==,propiedad ubicada gobernador alcorta cuadra co...,living chico cocina comedor amplio,,
7,ucluEQ4sNKNVbwyKJFnaBw==,casa venta calle alejandro magno hermosa vista...,completo living cocina comedor nivel,,
8,a/LC2bANkFuEJYuHglYSTg==,hermosa casa ph ubicado 10 cuadras centro vill...,bano comodo cocina comedor amoblada,subir 1 piso escalera compuesto,
9,XWCMzAub+1YSpvBCuREdPA==,hermosa casa ph ubicada barrio santa rita barr...,totalmente reciclado cocina integrada barra,,


In [111]:
Counter(" ".join(df_nostopwords["cocina"]).split()).most_common(30)

[('cocina', 7423),
 ('comedor', 3618),
 ('living', 1896),
 ('integrada', 1264),
 ('bano', 1082),
 ('lavadero', 658),
 ('separada', 623),
 ('muebles', 565),
 ('diario', 562),
 ('toilette', 531),
 ('balcon', 493),
 ('completo', 476),
 ('amplia', 399),
 ('independiente', 397),
 ('mesada', 338),
 ('2', 334),
 ('completa', 323),
 ('barra', 313),
 ('amplio', 310),
 ('equipada', 308),
 ('dormitorio', 306),
 ('dormitorios', 302),
 ('salida', 275),
 ('semi', 271),
 ('banos', 267),
 ('espacio', 211),
 ('recepcion', 201),
 ('3', 197),
 ('frente', 196),
 ('1', 195)]

In [112]:
Counter(" ".join(df_nostopwords["piso"]).split()).most_common(30)

[('piso', 2772),
 ('pisos', 1811),
 ('frente', 468),
 ('porcelanato', 398),
 ('living', 332),
 ('ambientes', 301),
 ('escalera', 294),
 ('parquet', 287),
 ('comedor', 277),
 ('edificio', 266),
 ('madera', 249),
 ('3', 245),
 ('departamento', 243),
 ('2', 242),
 ('ubicado', 239),
 ('semipiso', 210),
 ('balcon', 193),
 ('cocina', 188),
 ('dormitorios', 178),
 ('bano', 175),
 ('alto', 169),
 ('excelente', 162),
 ('vista', 160),
 ('4', 159),
 ('luminoso', 157),
 ('contrafrente', 153),
 ('dormitorio', 145),
 ('calefaccion', 134),
 ('amplio', 125),
 ('entrepiso', 125)]

In [90]:
export = df_nostopwords["property_description"].head(5)
np.savetxt('export.txt', export.values, fmt='%s')
export.head()

0    corredor responsable maria alejandra genoveva ...
1    hermoso ph tipo casa 2 ambientes apto credito ...
2    excelente ph nunez 4 amb oportunidad 154 m2 to...
3    unico ph 4 dormitorios impecable hermoso tranq...
4    ph 2 amb villa crespo exclusivo profesional ex...
Name: property_description, dtype: object

In [91]:
# import nltk
# from nltk.collocations import *
# bigram_measures = nltk.collocations.BigramAssocMeasures()
# trigram_measures = nltk.collocations.TrigramAssocMeasures()
#
# # change this to read in your data
# finder = BigramCollocationFinder.from_words(
#     nltk.corpus.genesis.words(df_decrip["property_description"]))
#
# # only bigrams that appear 3+ times
# finder.apply_freq_filter(3)
#
# # return the 10 n-grams with the highest PMI
# finder.nbest(bigram_measures.pmi, 10)